<a href="https://colab.research.google.com/github/michelmiranda/Semantic-Web-Services/blob/master/Project_Recommend_Educational_Resources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Introdução**
# **Problemática**
# **Solução**
# **Objetivo**




In [0]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re

In [3]:

search_words = []

N_words = int(input("Digite a quantidade de palavras-chave que você deseja pesquisar: "))
x = 0
while x < N_words:
    search_word = str(input("Digite a palavra-chave nº" + str((x+1)) + " de pesquisa: "))
    search_words.append(search_word.replace(" ", "+"))
    x = x + 1

N_pages = int(input("Digite a quantidade de páginas do Youtube para se pesquisar: "))

Digite a quantidade de palavras-chave que você deseja pesquisar: 3
Digite a palavra-chave nº1 de pesquisa: POO Herança
Digite a palavra-chave nº2 de pesquisa: Herança JAVA POO
Digite a palavra-chave nº3 de pesquisa: JAVA HERANÇA
Digite a quantidade de páginas do Youtube para se pesquisar: 5


In [5]:

search_words

['POO+Herança', 'Herança+JAVA+POO', 'JAVA+HERANÇA']

In [6]:
N_pages

5

In [7]:
search_url = "https://www.youtube.com/results?search_query={}&p={}"
search_words

['POO+Herança', 'Herança+JAVA+POO', 'JAVA+HERANÇA']

In [0]:
urls_code = []
urls_searched = []
words_searched = []
for word in search_words:
    for page in range(1, (1+N_pages)):
        url = search_url.format(word, page)
        url_code = rq.get(url)
        words_searched.append(word)
        urls_searched.append(url)
        urls_code.append(url_code.text)

urls_searched = pd.Series(urls_searched)
urls_code = pd.Series(urls_code)

In [9]:
urls_code.head()


0      <!DOCTYPE html><html lang="en" data-cast-api...
1      <!DOCTYPE html><html lang="en" data-cast-api...
2      <!DOCTYPE html><html lang="en" data-cast-api...
3      <!DOCTYPE html><html lang="en" data-cast-api...
4      <!DOCTYPE html><html lang="en" data-cast-api...
dtype: object

In [11]:
dados_brutos = pd.DataFrame({"urls": urls_searched, "code": urls_code, "query": words_searched})
dados_brutos.to_csv("dados_brutos.txt", index=False)
dados_brutos.head()

,urls,code,query
0,https://www.youtube.com/results?search_query=P...,"<!DOCTYPE html><html lang=""en"" data-cast-api...",POO+Herança
1,https://www.youtube.com/results?search_query=P...,"<!DOCTYPE html><html lang=""en"" data-cast-api...",POO+Herança
2,https://www.youtube.com/results?search_query=P...,"<!DOCTYPE html><html lang=""en"" data-cast-api...",POO+Herança
3,https://www.youtube.com/results?search_query=P...,"<!DOCTYPE html><html lang=""en"" data-cast-api...",POO+Herança
4,https://www.youtube.com/results?search_query=P...,"<!DOCTYPE html><html lang=""en"" data-cast-api...",POO+Herança


In [12]:
titles = []
urls_videos = []
pages_video = []
queries = []

for n, page in enumerate(dados_brutos["code"].apply(BeautifulSoup)):
    for link in page.findAll("a"):
        if link.has_attr("aria-describedby") == True:
            queries.append(dados_brutos["query"].iloc[n])
            titles.append(link["title"])
            urlls_videos = "https://youtube.com.br" + link["href"]
            urls_videos.append(urlls_videos)
            pages_video.append(rq.get(urlls_videos).text)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:3848: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 3848 of the file /usr/local/lib/python3.6/dist-packages/pandas/core/series.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  mapped = lib.map_infer(values, f, convert=convert_dtype)


In [0]:
data = pd.DataFrame({"title": titles, "urls_videos": urls_videos, "query": queries, "pages": pages_video})

In [16]:
data.head()

,title,urls_videos,query,pages
0,Curso POO Teoria #10a - Herança (Parte 1),https://youtube.com.br/watch?v=_PZldwo0vVo,POO+Herança,"<!DOCTYPE html><html lang=""en"" data-cast-api-e..."
1,Curso POO Java #10b - Herança (Parte 1),https://youtube.com.br/watch?v=19IGAeoFKlU,POO+Herança,"<!DOCTYPE html><html lang=""en"" data-cast-api-e..."
2,Curso de POO Java (Programação Orientada a Obj...,https://youtube.com.br/watch?v=KlIL63MeyMY&lis...,POO+Herança,"<!DOCTYPE html><html lang=""en"" data-cast-api-e..."
3,Curso de Java 37 - Orientação a Objetos: Herança,https://youtube.com.br/watch?v=MOXLCjL4Ik4,POO+Herança,"<!DOCTYPE html><html lang=""en"" data-cast-api-e..."
4,Curso POO Teoria #11a - Herança (Parte 2),https://youtube.com.br/watch?v=He887D2WGVw,POO+Herança,"<!DOCTYPE html><html lang=""en"" data-cast-api-e..."


In [0]:
data.to_csv("dados_processados.csv", index=False)

In [0]:
gots_datas = []
for page in data["pages"]:
    parsed = BeautifulSoup(page, "html.parser")
    
    class_watch = parsed.find_all(attrs={"class":re.compile(r"watch")})
    id_watch = parsed.find_all(attrs={"id":re.compile(r"watch")})
    channel = parsed.find_all("a", attrs={"href":re.compile(r"channel")})
    meta = parsed.find_all("meta")
    
    got_data = {}
    
    for cl in class_watch:
        colname = "_".join(cl["class"])
        if "clearfix" in colname:
            continue
        got_data[colname] = cl.text.strip()
    
    for cl in id_watch:
        colname = cl["id"]
        got_data[colname] = cl.text.strip()
        
    for cl in meta:
        colname = cl.get("property")
        if colname is not None:
            got_data[colname] = cl["content"]
    
    for n, cl in enumerate(channel):
        got_data["channel_link_{}".format(n)] = cl["href"]
    
    gots_datas.append(got_data)

In [0]:
adapt_dict= {}
for colname in gots_datas[0].keys():
    adapt_dict[colname] = []

In [0]:
all_dfs = list(pd.Series(gots_datas).apply(lambda x: pd.DataFrame(x, index=range(1))))

In [0]:
videos_data = pd.concat(all_dfs).reset_index(drop=True)

In [22]:
videos_data.head()

,_content-alignment_watch-small_,watch-playlist_player-height,watch-queue-header,watch-queue-info,watch-queue-info-icon,watch-queue-title,watch-queue-control-bar_control-bar-button,watch-queue-mole-info,watch-queue-control-bar-icon,watch-queue-icon_yt-sprite,watch-queue-title-container,watch-queue-count,watch-queue-menu_yt-uix-button-menu_yt-uix-button-menu-dark-overflow-action-menu_hid,watch-queue-menu-choice_overflow-menu-choice_yt-uix-button-menu-item,watch-queue-controls,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_prev-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-prev_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_play-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-play_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_pause-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_hid_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-pause_yt-sprite,yt-uix-button_yt-uix-button-size-default_yt-uix-button-empty_yt-uix-button-has-icon_control-bar-button_next-watch-queue-button_yt-uix-button-opacity_yt-uix-tooltip_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-queue-next_yt-sprite,watch-queue-items-container_yt-scrollbar-dark_yt-scrollbar,watch-queue-items-list,_content-alignment_watch-player-playlist_,watch-main-col_,watch-title-container,watch-title,watch-secondary-actions_yt-uix-button-group,watch-view-count,watch-action-panels_yt-uix-button-panel_hid_yt-card_yt-card-has-padding,watch-time-text,watch-extras-section,watch-meta-item_yt-uix-expander-body,content_watch-info-tag-list,watch-sidebar,watch-playlist_player-height_hid,watch-sidebar-gutter_yt-card_yt-card-has-padding_yt-uix-expander_yt-uix-expander-collapsed,watch-sidebar-section,...,watch-channel-brand-div,watch-channel-brand-div-text,watch7-sidebar-modules,watch-related,shared-addto-watch-later-login,og:site_name,og:url,og:title,og:image,og:image:width,og:image:height,og:description,al:ios:app_store_id,al:ios:app_name,al:ios:url,al:android:url,al:android:app_name,al:android:package,al:web:url,og:type,og:video:url,og:video:secure_url,og:video:type,og:video:width,og:video:height,og:video:tag,fb:app_id,channel_link_0,yt-uix-button_yt-uix-button-size-default_yt-uix-button-player-controls_yt-uix-button-empty_yt-uix-button-has-icon_no-icon-markup_yt-uix-playlistlike_watch-playlist-like_yt-uix-button-opacity_yt-uix-tooltip,yt-uix-button-icon_yt-uix-button-icon-watch-appbar-autoplay_yt-sprite,yt-uix-button-icon_yt-uix-button-icon-watch-appbar-shuffle-video-list_yt-sprite,yt-uix-button-icon_yt-uix-button-icon-watch-appbar-play-prev_yt-sprite,yt-uix-button-icon_yt-uix-button-icon-watch-appbar-play-next_yt-sprite,watch-playlist_player-height_,watch-appbar-playlist,channel_link_1,channel_link_2,watch-meta-item_,watch-sidebar-live-chat,channel_link_3
0,This video is unavailable.\n\n \n\n\n\n\n\n...,Watch QueueQueueWatch QueueQueue \nRemove allD...,Watch QueueQueueWatch QueueQueue \nRemove allD...,Watch QueueQueue,,Watch Queue,Watch QueueQueue \nRemove allDisconnect,Watch QueueQueue,,,Watch QueueQueue,,Remove allDisconnect,Disconnect,,,,,,,,,,Loading...,Loading...,,"{\n ""@context"": ""http://schema.org"",\n ""...",Curso POO Teoria #10a - Herança (Parte 1),Curso POO Teoria #10a - Herança (Parte 1),Add to\n\nWant to watch this again later?\n\n ...,"87,572 views",Loading...\n \n\n\n\n\n\n\n\n\n\n\n\nLoadin...,"Published on Oct 24, 2016",Category\n \n\nEducation,Category\n \n\nEducation,Education,Advertisement\n \n\n\n\n\n\n\n\n\nAutopla...,,Advertisement\n \n\n\n\n\n\n\n\n\nAutopla...,Curso POO Teoria #11a - Herança (Parte 2)\n \...,...,Advertisement,Advertisement,"Autoplay\n\n\nWhen autoplay is enabled, a sugg...",Curso POO

In [0]:
selected_columns = ["watch-title", "watch-view-count", "watch-time-text", "content_watch-info-tag-list", "watch7-headline",
                   "watch7-user-header", "watch8-sentiment-actions", "og:image", "og:image:width", "og:image:height",
                   "og:description", "og:video:width", "og:video:height", "og:video:type", "channel_link_0"]

In [0]:
videos_data[selected_columns].to_csv("features_selecionadas(sem labels).csv")

In [28]:
videos_data[selected_columns].dropna()

,watch-title,watch-view-count,watch-time-text,content_watch-info-tag-list,watch7-headline,watch7-user-header,watch8-sentiment-actions,og:image,og:image:width,og:image:height,og:description,og:video:width,og:video:height,og:video:type,channel_link_0
0,Curso POO Teoria #10a - Herança (Parte 1),"87,572 views","Published on Oct 24, 2016",Education,Curso POO Teoria #10a - Herança (Parte 1),Curso em Vídeo\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoad...,"87,572 views\n\n\n\n\n\n\n\n4,361\n\nLike this...",https://i.ytimg.com/vi/_PZldwo0vVo/maxresdefau...,1280,720,"Nessa aula de POO, vamos aprender o que é Hera...",1280,720,text/html,/channel/UCrWvhVmt0Qac3HgsjQK62FQ
1,Curso POO Java #10b - Herança (Parte 1),"74,017 views","Published on Oct 24, 2016",Education,Curso POO Java #10b - Herança (Parte 1),Curso em Vídeo\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoad...,"74,017 views\n\n\n\n\n\n\n\n3,408\n\nLike this...",https://i.ytimg.com/vi/19IGAeoFKlU/maxresdefau...,1280,720,"Nessa aula de POO, vamos aprender a aplicar o ...",1280,720,text/html,/channel/UCrWvhVmt0Qac3HgsjQK62FQ
2,Course OOP Theory # 01a - What is Object Orien...,"822,461 views","Published on Aug 1, 2016",Education,Course OOP Theory # 01a - What is Object Orien...,Curso em Vídeo\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoad...,"822,461 views\n\n\n\n\n\n\n\n30,388\n\nLike th...",https://i.ytimg.com/vi/KlIL63MeyMY/maxresdefau...,1280,720,"In this OOP class, we will learn what is Objec...",1280,720,text/html,/channel/UCrWvhVmt0Qac3HgsjQK62FQ
3,Curso de Java 37 - Orientação a Objetos: Herança,"27,130 views","Published on Jan 25, 2016",Education,Curso de Java 37 - Orientação a Objetos: Herança,Loiane Groner\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoadi...,"27,130 views\n\n\n\n\n\n\n\n1,206\n\nLike this...",https://i.ytimg.com/vi/MOXLCjL4Ik4/maxresdefau...,1280,720,Curso de Java Completo com Certificado e Fórum...,960,720,text/html,/channel/UCqQn92noBhY9VKQy4xCHPsg
4,Curso POO Teoria #11a - Herança (Parte 2),"77,842 views","Published on Oct 31, 2016",Education,Curso POO Teoria #11a - Herança (Parte 2),Curso em Vídeo\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoad...,"77,842 views\n\n\n\n\n\n\n\n4,686\n\nLike this...",https://i.ytimg.com/vi/He887D2WGVw/maxresdefau...,1280,720,"Nessa aula de POO, vamos aprender como funcion...",1280,720,text/html,/channel/UCrWvhVmt0Qac3HgsjQK62FQ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Aula 03 - Associação e herança - Curso de Prog...,"15,992 views","Published on Apr 21, 2016",Education,Aula 03 - Associação e herança - Curso de Prog...,Leandro Guarino\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoa...,"15,992 views\n\n\n\n\n\n\n\n498\n\nLike this v...",https://i.ytimg.com/vi/6FJ5usl7uTw/hqdefault.jpg,480,360,Aprenda a identificar os relacionamentos entre...,480,360,text/html,/channel/UC7Q_uLJ5hpA3NzlP7I4dGqQ
132,POO- Herança em Java,3 views,"Published on Sep 9, 2018",People & Blogs,POO- Herança em Java,Yago Nunes\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoading....,3 views\n\n\n\n\n\n\n\n0\n\nLike this video?\n...,https://i.ytimg.com/vi/AXYpmsBJvpU/maxresdefau...,1280,720,,1280,720,text/html,/channel/UCYW5BhaAcDgHQ6-Tw684AkA
133,Programação Orientada a Objetos - Herança em Java,2 views,"Published on Aug 29, 2018",People & Blogs,Programação Orientada a Objetos - Herança em Java,Yago Nunes\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoading....,2 views\n\n\n\n\n\n\n\n1\n\nLike this video?\n...,https://i.ytimg.com/vi/7ECVtIU_kNA/maxresdefau...,1280,720,,1280,720,text/html,/channel/UCYW5BhaAcDgHQ6-Tw684AkA
134,Exercício Java 4 - Herança,774 views,"Published on Feb 12, 2017",Science & Technology,Exercício Java 4 - Herança,Adriano Lima\n\n\n\n\n\n\n\n\n\n\n\n\n\nLoadin...,774 views\n\n\n\n\n\n\n\n4\n\nLike this video?...,https://i.ytimg.com/vi/2fPONQmKq78/maxresdefau...,1280,720,"Série de exercícios em Java, abordando diverso...",960,720,text/html,/channel/UCT51I1SMdqc5e5gA4pgOTSg
